Sta 663 - Statistical Computing and Computation - Midterm 2
-----------
Due Wednesday, April 20th by 5:00 pm.



## Setup

In [1]:
# Load necessary packages
import torch

## Task 1 - `newton()` function

This task completes the function of newton optimization. Firstly, convert all inputs to Tensors type and get the initial value for function. Before running newton method, check if all initials including objective and derivatives are finite, if one of them is not finite, then raise ValueError. Afterwards, creating while loop to iterate each step and set the convergent boolean value to ensure the final result is convergent. In this while loop, first, we should get the Jacobian and Hessian value for updated thetas, and then we could get step to update the further thetas and the output of function. Note that here we should check if hessian is positive definite: I define a new function `check_psd` to check, if it's positive definite, then return true, otherwise, false. Plus, if the hessian is not positive definite, then we should start by adding epsilon and keep multiplying epsilon by 10 until it is. Furthermore, we could move to the max_half loop inside maxit loop to get the new thetas and function results. Note here if reduction does not going well or the newton method is not converged, we should repeat half the step. If it's still fail to reduce the objective, then should raise 'step fails' error. If above is running well, we could store our new values in order to get the minimum at the end. Meanwhile, the most important task is to judge whether the gradient vector is close enough to zero by max(abs(g)) < (abs(f0)+fscale)*tol, and if it satisfies this condition, convergent will convert to True, otherwise, should raise the error after trying maxit. Finally, collecting all the necessary results to a dictionary. 

In [2]:
def newton(theta, f, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 20):
    """
    Return a dictionary containing:
    f -- the value of the objective function at the minimum
    theta -- the value of the parameters at the minimum
    iter -- the number of iterations taken to reach the minimum
    grad -- the gradient vector at the minimum
    
    Keyword arguments:
    theta -- a vector of initial values for the optimization parameters
    f -- the objective function to minimize
    tol -- the convergence tolerance
    fscale -- a rough estimate of the magnitude of f at the optimum 
    maxit -- the maximum number of Newton iterations to try before giving up
    max_half -- the maximum number of times a step should be halved before concluding that the step has failed to improve the objective
    """
        
    # convert inputs to torch, create the list of inputs, and get the first input of function 
    theta_i = torch.tensor(theta, dtype = torch.double)
    tol = torch.tensor(tol, dtype = torch.double)
    fscale = torch.tensor(fscale, dtype = torch.double)
    f_i = f(theta)
    
    f_i_list = []
    theta_i_list = []
    f_i_list.append(f_i)
    theta_i_list.append(theta_i)
    
    y_0 = f(theta_i)
    j_0 = torch.autograd.functional.jacobian(f, theta_i)
    
    # check if all initials, objective function and its derivative are finite
    if torch.all(torch.isfinite(theta_i)) == True and torch.all(torch.isfinite(y_0)) == True and torch.all(torch.isfinite(j_0)) == True:
        # iterate each step
        i = 0
        convergent = False
        iterate = 1
        while i < maxit and convergent == False:
            j_i = torch.autograd.functional.jacobian(f, theta_i_list[-1])
            h_i = torch.autograd.functional.hessian(f, theta_i_list[-1])
            
            # check if hessian is PD, if not, then perturbe Hessian
            # define a function to check positive definite
            def check_psd(A):
                try:
                    torch.linalg.cholesky(A)
                    return True
                except torch.linalg.LinAlgError:
                    return False
            k = 0
            while (check_psd(h_i) == False):
                epsilon = 10**(k)*torch.max(torch.abs(h_i))*1e-8*torch.eye(h_i.shape[1])
                h_i = h_i + epsilon
                # check if finite
                if torch.any(torch.isinf(h_i)) == True:
                    raise Exception("ValueError: Hessian is not finite")
                # add more epsilon
                k = k+1
                
            # get step
            step = -torch.linalg.solve(h_i, j_i)
            
            # iterate until max_half
            for j in range(max_half):
                theta_i_new = theta_i_list[-1] + step
                f_i_new = f(theta_i_new)
                
                # check if reduce well or the newton method has converged, if not, repeatedly half the step
                if (f_i_new < f_i_list[-1]) | (torch.max(torch.abs(j_i)) < (torch.abs(f_i_new)+fscale)*tol):
                    break
                step /= 2
            
            # check if step fails after max_half
            if (f_i_new >= f_i_list[-1]) & ((torch.max(torch.abs(j_i)) > (torch.abs(f_i_new)+fscale)*tol)):
                raise Exception("ValueError: step fails") 
                  
            # get new and store
            f_i_list.append(f_i_new)
            theta_i_list.append(theta_i_new)
   
            # check if current is convergent 
            if max(abs(j_i)) < (abs(f_i_new)+fscale)*tol:
                convergent = True
                break
                
            i += 1
            iterate += 1
            
        # final result
        grad = torch.autograd.functional.jacobian(f, theta_i_new)
        result = {
            "f": f_i_new,
            "theta": theta_i_new,
            "iter": iterate,
            "grad": grad
        }
             
        if i >= maxit and convergent == False:
            raise Exception("ValueError: maxit is reached without convergence")
    else:
        raise Exception("ValueError: the objective or derivatives are not finite at the initials")

    return result

## Task 2 - Minimization examples


### 1. Quadratic function


In [3]:
## Objective function implementation
def quadratic(theta):
    x,y = theta
    return x**2 - 2*x + 2*(y**2) + y + 3

In [4]:
## theta 1 minimization
newton([20,30], quadratic, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [5]:
## theta 2 minimization
newton([-100,-200], quadratic, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [6]:
## theta 3 minimization
newton([-1,1], quadratic, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 8)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0.], dtype=torch.float64)}

### 2. Rosenbrock's function

In [7]:
## Objective function implementation
def Rosenbrock(theta):
    x,y = theta
    return 10 * (y - x**2)**2 + (1 - x)**2

In [8]:
## theta 1 minimization
newton([0.3,0.5], Rosenbrock, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([1., 1.], dtype=torch.float64),
 'iter': 6,
 'grad': tensor([-0., 0.], dtype=torch.float64)}

In [9]:
## theta 2 minimization
newton([20,30], Rosenbrock, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([1., 1.], dtype=torch.float64),
 'iter': 38,
 'grad': tensor([-0., 0.], dtype=torch.float64)}

In [10]:
## theta 3 minimization
newton([200,300], Rosenbrock, tol = 1e-8, fscale = 1.0, maxit = 200, max_half = 20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([1., 1.], dtype=torch.float64),
 'iter': 157,
 'grad': tensor([-0., 0.], dtype=torch.float64)}


### 3. Poisson regression likelihood

In [11]:
x = [
   0.11, -0.06, -0.96, -0.48, -0.59, -0.42, -0.15,  1.14, 0.94, 
  -0.86, -0.08,  1.00, -2.01,  2.17, -0.20,  0.82, -0.13, 0.26, 
   0.22,  1.05
]

y = [4, 2, 4, 1, 1, 3, 4, 5, 7, 3, 5, 7, 0, 4, 2, 7, 3, 3, 2, 8]

In [12]:
## Objective function implementation
def PoissonRegression(theta):
    beta_0, beta_1 = theta
    lambda_i = beta_0 + beta_1 * torch.tensor(x)
    likelihood = torch.sum(torch.tensor(y)*lambda_i-torch.exp(lambda_i)-torch.log(torch.tensor([torch.jit._builtins.math.factorial(i) for i in y])))
    return -likelihood

In [13]:
## theta 1 minimization
newton([15,20], PoissonRegression, tol = 1e-4, fscale = 30.0, maxit = 100, max_half = 20)

{'f': tensor(37.8802),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 60,
 'grad': tensor([-2.3842e-06, -1.1921e-06], dtype=torch.float64)}

In [14]:
## theta 2 minimization
newton([1,4], PoissonRegression, tol = 1e-4, fscale = 30.0, maxit = 100, max_half = 20)

{'f': tensor(37.8802),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 11,
 'grad': tensor([-2.3842e-06,  5.9605e-07], dtype=torch.float64)}

In [15]:
## theta 3 minimization
newton([7,7], PoissonRegression, tol = 1e-4, fscale = 30.0, maxit = 100, max_half = 20)

{'f': tensor(37.8802),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 23,
 'grad': tensor([-2.1458e-06,  1.1921e-07], dtype=torch.float64)}